In [ ]:
from google.colab.drive import mount
mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import nltk
import hazm
from hazm import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from catboost import CatBoostClassifier as cbc

In [ ]:
data = pd.read_csv('/content/drive/My Drive/files/per.csv')

In [ ]:
with open('/content/drive/My Drive/files/stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)

In [ ]:
stemmer = hazm.Stemmer()

In [ ]:
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', '')
    }

In [ ]:
vectorizer = TfidfVectorizer()

vectorizer.fit(dataset['title_body'])
X = vectorizer.transform(dataset['title_body'])

y = dataset['category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model = cbc(iterations=1000 , verbose=200 , learning_rate=1 , random_state=1441 , task_type='GPU')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

0:	learn: 1.9471195	total: 598ms	remaining: 9m 57s
200:	learn: 0.8759606	total: 45.4s	remaining: 3m
400:	learn: 0.6727277	total: 1m 26s	remaining: 2m 9s
600:	learn: 0.5290519	total: 2m 7s	remaining: 1m 24s
800:	learn: 0.3942545	total: 2m 48s	remaining: 41.9s
999:	learn: 0.3084108	total: 3m 29s	remaining: 0us
              precision    recall  f1-score   support

      آموزشي       0.79      0.81      0.80       271
     اجتماعي       0.43      0.46      0.45       254
     اقتصادي       0.66      0.69      0.68       239
     بهداشتي       0.86      0.86      0.86       243
      تاريخي       0.79      0.86      0.83       228
       سياسي       0.67      0.66      0.66       231
        علمي       0.70      0.70      0.70       287
      فرهنگي       0.79      0.74      0.76       285
  فقه و حقوق       0.82      0.73      0.77       248
       مذهبي       0.86      0.83      0.84       221
       ورزشي       0.90      0.88      0.89       243

    accuracy                           0